# 用Python从头开始搭建人工神经网络

## 层与层

首先，大框架如下：

1. 通过**输入层**向人工神经网络输入数据。
2. 数据在网络中**一层接着一层**“流动”，直到输出。
3. 一旦得到了输出，可以计算**误差的标量**。
4. 最终可以通过减去误差对参数的**导数**来调整给定的参数（权重或偏差）。
5. 步骤4可以迭代进行。

最重要的步骤是第**4**步。构建的网络可以任意多层，每层有任意的形式。但如果修改、添加、或者删除网络中的某一层，网络输出就会改变，也就会改变误差，也就会改变误差对参数的导数。所以，需要能够在不考虑网络结构的情况下计算导数，包括不考虑激活函数和损失函数。

为了实现以上目的，需要每一层单独处理。

## 每一层需要实现什么

每一层（全连接层、卷积、最大池化、dropout层等）我们都至少需要创建2样共通的东西，即**输入**和**输出**数据。

$$
\mathbf{X}\rightarrow\boxed{\text{layer}}\rightarrow\mathbf{Y}
$$

### 前向传播

要点：**上一层的输出是下一层的输入**。

$$
\stackrel{\text{X}}{\longrightarrow}\boxed{\text{Layer 1}}\stackrel{\text{H}_1}{\longrightarrow}\boxed{\text{Layer 2}}\stackrel{\text{H}_2}{\longrightarrow}\boxed{\text{Layer 3}}\stackrel{\text{Y, E}}{\longrightarrow}
$$

如上所示，将数据输入第一层，每一层的输出都成为下一层的输入，直到到达网络末端。比较网络输出结果（$Y$）与预期结果（$Y^*$），可以计算误差$\mathbf{E}$。目标是通过反向传播算法来调整网络中的参数，以**最小化**误差。

### 梯度下降

通过调整网络中的参数（$\mathbf{w}$）使得总误差$\mathbf{E}$**减小**：
$$
w\leftarrow w-\alpha\frac{\partial{E}}{\partial{w}}
$$
**学习率$\alpha$**的范围是[0,1]。注意**$\partial{E}/\partial{w}$**（$E$对$w$的导数）。**需要能够为网络的任何参数找到该表达式的值，而不管其架构如何。**

### 反向传播

假设已知一层的**输出误差对输出的导数**（$\partial{E}/\partial{Y}$），就应该能够计算**误差对输入的导数**（$\partial{E}/\partial{X}$）。
$$
\frac{\partial{E}}{\partial{X}}\leftarrow\boxed{\text{layer}}\leftarrow\frac{\partial{E}}{\partial{Y}}
$$
记住，`E`是**标量**（一个数），而`X`和`Y`是**矩阵**。
$$
\begin{align*}
\frac{\partial{E}}{\partial{X}}&=\left[\frac{\partial{E}}{\partial{x_1}}\quad\frac{\partial{E}}{\partial{x_2}}\quad\cdots\quad\frac{\partial{E}}{\partial{x_i}}\right]\\
\frac{\partial{E}}{\partial{Y}}&=\left[\frac{\partial{E}}{\partial{y_1}}\quad\frac{\partial{E}}{\partial{y_2}}\quad\cdots\quad\frac{\partial{E}}{\partial{y_i}}\right]
\end{align*}
$$
先暂时不用理会$\partial{E}/\partial{X}$。有一个小技巧，可以通过$\partial{E}/\partial{Y}$很容易计算得出$\partial{E}/\partial{W}$（如果该层有可训练参数），而**无需任何网络结构的信息！**即，使用链式法则：
$$
\frac{\partial{E}}{\partial{w}}=\sum_j\frac{\partial{E}}{\partial{y_j}}\frac{\partial{y_j}}{\partial{w}}
$$
未知数$\partial{y_j}/\partial{w}$完全取决于该层如何计算输出。所以如果每层都能计算$\partial{E}/\partial{Y}$（$Y$是该层输出），就可以更新该层的参数！

### 为什么要算$\partial{E}/\partial{X}$?

不要忘记，上一层的输出是下一层的输入。这就意味着某一层的$\partial{E}/\partial{X}$就是其上一层的$\partial{E}/\partial{Y}$！这就是误差反向传播！再次利用链式法则：
$$
\frac{\partial{E}}{\partial{x_i}}=\sum_j\frac{\partial{E}}{\partial{y_j}}\frac{\partial{y_j}}{\partial{x_i}}
$$
上面这个公式是理解反向传播的关键！通过这个公式，就能够从头开始构建整个深度人工神经网络！

### 图解反向传播

下图中，第3层使用$\partial{E}/\partial{Y}$来更新参数，然后将$\partial{E}/\partial{H_2}$通过$\partial{Y}/\partial{H_2}$传递给上一层。第二层又重复同样操作，直到输入层。

![backpropagation](./images/backpropagation.jpg)

### 抽象基类：层

抽象类*层*，所有其他类型的层都将继承它，用于处理**输入**，**输出**，**前向**和**反向**传播方法。

In [1]:
# Base class
class Layer:
    def __init__(self) -> None:
        self.input = None
        self.output = None
    
    # computes the output Y of a layer for a given input X
    def forward_propagation(self, input):
        raise NotImplementedError
    
    # computes dE/dX for a given dE/dY (and update parameters if any)
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

## 全连接层

首先定义和实现第一种类型的层：全连接层（Fully Connected, FC）。FC层是最基础的层类型，所有输入神经元与所有输出神经元相连。

![fully connected layer](./images/fully_connected_layer.jpg)

### 前向传播

每个输出神经元的值可根据下式计算：
$$
y_j=b_j+\sum_i{x_i w_{ij}}
$$
可以使用矩阵**点乘**来简单表示：
$$
X=\left[x_1\quad\cdots\quad x_i\right]\quad
W=\begin{bmatrix}
w_{11} & \cdots & w_{1j}\\
\vdots & \ddots & \vdots\\
w_{i1} & \cdots & w_{ij}
\end{bmatrix}\quad
B=\left[b_1\quad\cdots\quad b_j\right]
$$
$$
Y=XW+B
$$

### 反向传播
假设*可以计算某一层的误差对**该层输出**的导数矩阵（$\partial{E}/\partial{Y}$）*。还需要：

1. 误差对参数的导数（$\partial{E}/\partial{W}$，$\partial{E}/\partial{B}$）
2. 误差对输入的导数（$\partial{E}/\partial{X}$）

先计算$\partial{E}/\partial{W}$，该矩阵与$W$的维度一样：`i x j`，`i`是输入神经元数量，`j`是输出神经元数量。对每一个权重计算梯度：
$$
\frac{\partial{E}}{\partial{W}}=
\begin{bmatrix}
\frac{\partial{E}}{\partial{w_{11}}} & \cdots & \frac{\partial{E}}{\partial{w_{1j}}} \\
\vdots & \ddots & \vdots \\
\frac{\partial{E}}{\partial{w_{i1}}} & \cdots & \frac{\partial{E}}{\partial{w_{ij}}}
\end{bmatrix}
$$
利用链式法则有：
$$
\begin{split}
\frac{\partial{E}}{\partial{w_{ij}}}&=\frac{\partial{E}}{\partial{y_1}}\frac{\partial{y_1}}{\partial{w_{ij}}}+\cdots+\frac{\partial{E}}{\partial{y_j}}\frac{\partial{y_j}}{\partial{w_{ij}}}\\
&=\frac{\partial{E}}{\partial{y_j}}x_i
\end{split}
$$
因此，
$$
\begin{split}
\frac{\partial{E}}{\partial{W}}&=
\begin{bmatrix}
\frac{\partial{E}}{\partial{y_1}}x_1 & \cdots & \frac{\partial{E}}{\partial{y_j}}x_1 \\
\vdots & \ddots & \vdots \\
\frac{\partial{E}}{\partial{y_1}}x_i & \cdots & \frac{\partial{E}}{\partial{y_j}}x_i
\end{bmatrix} \\
&=
\begin{bmatrix}
x_1 \\
\vdots \\
x_i
\end{bmatrix}
\left[\frac{\partial{E}}{\partial{y_1}}\quad\cdots\quad\frac{\partial{E}}{\partial{y_j}}\right] \\
&=X^t\frac{\partial{E}}{\partial{Y}}
\end{split}
$$
上面是用来更新权重的第一个公式。下面计算$\frac{\partial{E}}{\partial{B}}$。
$$
\frac{\partial{E}}{\partial{B}}=\left[\frac{\partial{E}}{\partial{b_1}}\quad\frac{\partial{E}}{\partial{b_2}}\quad\cdots\quad\frac{\partial{E}}{\partial{b_j}}\right]
$$
再次，$\frac{\partial{E}}{\partial{B}}$必须与$B$本身维度相同，一个偏差一个梯度值。再次利用链式法则：
$$
\begin{split}
\frac{\partial{E}}{\partial{b_j}}&=\frac{\partial{E}}{\partial{y_1}}\frac{\partial{y_1}}{\partial{b_j}}+\cdots+\frac{\partial{E}}{\partial{y_j}}\frac{\partial{y_j}}{\partial{b_j}}\\
&=\frac{\partial{E}}{\partial{y_j}}
\end{split}
$$
也就是，
$$
\begin{split}
\frac{\partial{E}}{\partial{B}}&=\left[\frac{\partial{E}}{\partial{y_1}}\quad\frac{\partial{E}}{\partial{y_2}}\quad\cdots\quad\frac{\partial{E}}{\partial{y_j}}\right]\\
&=\frac{\partial{E}}{\partial{Y}}
\end{split}
$$

现在已经计算了$\partial{E}/\partial{W}$和$\partial{E}/\partial{B}$，还剩下$\partial{E}/\partial{X}$这一个**非常重要**的项，因为它其实“充当”上一层的$\partial{E}/\partial{Y}$。
$$
\frac{\partial{E}}{\partial{X}}=\left[\frac{\partial{E}}{\partial{x_1}}\quad\frac{\partial{E}}{\partial{x_2}}\quad\cdots\quad\frac{\partial{E}}{\partial{x_i}}\right]
$$
再次利用链式法则：
$$
\begin{split}
\frac{\partial{E}}{\partial{x_i}}&=\frac{\partial{E}}{\partial{y_1}}\frac{\partial{y_1}}{\partial{x_i}}+\cdots+\frac{\partial{E}}{\partial{y_j}}\frac{\partial{y_j}}{\partial{x_i}}\\
&=\frac{\partial{E}}{\partial{y_1}}w_{i1}+\cdots+\frac{\partial{E}}{\partial{y_j}}w_{ij}
\end{split}
$$
最终，可以计算整个矩阵：
$$
\begin{split}
\frac{\partial{E}}{\partial{X}}&=\left[\left(\frac{\partial{E}}{\partial{y_1}}w_{11}+\cdots+\frac{\partial{E}}{\partial{y_j}}w_{1j}\right)\quad\cdots\quad\left(\frac{\partial{E}}{\partial{y_1}}w_{i1}+\cdots+\frac{\partial{E}}{\partial{y_j}}w_{ij}\right)\right]\\
&=\left[\frac{\partial{E}}{\partial{y_1}}\quad\cdots\quad\frac{\partial{E}}{\partial{y_j}}\right]
\begin{bmatrix}
w_{11} & \cdots & w_{i1} \\
\vdots & \ddots & \vdots \\
w_{1j} & \cdots & w_{ij}
\end{bmatrix} \\
&=\frac{\partial{E}}{\partial{Y}}W^t
\end{split}
$$

现在我们有了FC层计算所需的全部三个方程！
$$
\begin{align*}
\frac{\partial{E}}{\partial{X}}&=\frac{\partial{E}}{\partial{Y}}W^t\\
\frac{\partial{E}}{\partial{W}}&=X^t\frac{\partial{E}}{\partial{Y}}\\
\frac{\partial{E}}{\partial{B}}&=\frac{\partial{E}}{\partial{Y}}
\end{align*}
$$

### 全连接层代码

In [2]:
import numpy as np

# inherit from base class Layer
class FCLayer(Layer):
    # input_size = number of input neurons
    # output_size = number of output neurons
    def __init__(self, input_size, output_size) -> None:
        super().__init__()
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5
    
    # returns output for a iven input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output
    
    # computes dE/dW, dE/dB for a given output_error = dE/dY. Returns input_error = dE/dx.
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        # dBias = output_error

        # update parameters
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

### 激活层
到目前为止，所做的所有计算都是完全线性的。用这种模型学任何东西都是没有希望的。需要通过将非线性函数应用于某些层的输出，将**非线性**添加到模型中。

现在，我们需要为这种新类型的层重做整个过程！

不用担心，由于没有*可学习*的参数，速度会快得多。我们只需要计算一下$\partial{E}/\partial{X}$。

将激活函数及其导数分别称为`f`和`f'`。

#### 前向传播

非常直观，对于给定的输入`X`，输出只是简单将激活函数应用于`X`的每个元素。这意味着**输入**和**输出**具有**相同维度**。
$$
\begin{split}
Y&=\left[f(x_1)\quad\cdots\quad f(x_i)\right]\\
&=f(X)
\end{split}
$$

#### 反向传播

$$
\begin{split}
\frac{\partial{E}}{\partial{X}}&=\left[\frac{\partial{E}}{\partial{x_1}}\quad\cdots\quad\frac{\partial{E}}{\partial{x_i}}\right]\\
&=\left[\frac{\partial{E}}{\partial{y_1}}\frac{\partial{y_1}}{\partial{x_1}}\quad\cdots\quad\frac{\partial{E}}{\partial{y_i}}\frac{\partial{y_i}}{\partial{x_i}}\right]\\
&=\left[\frac{\partial{E}}{\partial{y_1}}f'(x_1)\quad\cdots\quad\frac{\partial{E}}{\partial{y_i}}f'(x_i)\right]\\
&=\left[\frac{\partial{E}}{\partial{y_1}}\quad\cdots\quad\frac{\partial{E}}{\partial{y_i}}\right]\cdot\left[f'(x_1)\quad\cdots\quad f'(x_i)\right]\\
&=\frac{\partial{E}}{\partial{Y}}\cdot f'(X)
\end{split}
$$
注意，这里我们使用两个矩阵之间的**元素相乘**（而在之前的公式中是点积）。

#### 激活函数代码

In [3]:
# inherit from base class Layer
class ActivationLayer(Layer):
    def __init__(self, activation, activation_prime) -> None:
        super().__init__()
        self.activation = activation
        self.activation_prime = activation_prime
    
    # returns the activated input
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output
    
    # Returns input_error = dE/dX for a given output_error = dE/dY.
    # learning_rate is not used because there is no "learnable" parameters.
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

单独编写一些激活函数及其导数。这些将在以后用于创建`ActivationLayer`。

In [4]:
import numpy as np

# activation function and its derivative
def tanh(x):
    return np.tanh(x)


def tanh_prime(x):
    return 1 - np.tanh(x)**2

### 损失函数

到目前为止，我们都假设$\partial{E}/\partial{Y}$已知（通过下一层）。但最后一层发生了什么？是怎么计算得到$\partial{E}/\partial{Y}$的？取决于如何计算误差。

网络错误**人为定义**，它衡量网络对给定输入数据表现得好坏。定义误差的方法有很多种，其中一种最常用的方法叫做**MSE——均方误差**。
$$
E=\frac{1}{n}\sum_i^n\left(y_i^*-y_i\right)^2
$$
其中`y*`和`y`分别代表**期望输出**和**实际输出**。可以把损失函数看作是最后一层，它把所有的输出神经元压缩成一个。我们现在需要对于每一层定义$\frac{\partial E}{\partial Y}$。
$$
\begin{split}
\frac{\partial{E}}{\partial{Y}}&=\left[\frac{\partial{E}}{\partial{y_1}}\quad\cdots\quad\frac{\partial{E}}{\partial{y_i}}\right]\\
&=\frac{2}{n}\left[y_1-y_1^*\quad\cdots\quad y_i-y_i^*\right]\\
&=\frac{2}{n}(Y-Y^*)
\end{split}
$$

In [5]:
import numpy as np

# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))


def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_true.size

### 网络类

In [6]:
class Network:
    def __init__(self) -> None:
        self.layers = []
        self.loss = None
        self.loss_prime = None

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)
    
    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime
    
    # predict output for given input
    def predict(self, input_data):
        # sample dimension first
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
        
        return result
    
    # train the network
    def fit(self, x, y, epochs, learning_rate):
        # sample dimension first
        samples = len(x)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                
                # compute loss (for display purpose only)
                err += self.loss(y[j], output)

                # backward propagation
                error = self.loss_prime(y[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
            
            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d    error=%f' % (i+1, epochs, err))

### 构建网络

终于我们可以使用我们的类创建一个神经网络，它可以是任意多层！我们将构建两个神经网络：一个简单的**XOR**和一个**MNIST**求解器。

#### 解XOR

从XOR开始总是很重要的，因为它是判断网络是否在学习任何东西的简单方法。

In [7]:
import numpy as np

# training data
x_train = np.array([[[0, 0]], [[0, 1]], [[1, 0]], [[1, 1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# network
net = Network()
net.add(FCLayer(2, 3))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(3, 1))
net.add(ActivationLayer(tanh, tanh_prime))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)

# test
out = net.predict(x_train)
print(out)

epoch 1/1000    error=0.367800
epoch 2/1000    error=0.325306
epoch 3/1000    error=0.311020
epoch 4/1000    error=0.304459
epoch 5/1000    error=0.300676
epoch 6/1000    error=0.298169
epoch 7/1000    error=0.296361
epoch 8/1000    error=0.294981
epoch 9/1000    error=0.293885
epoch 10/1000    error=0.292989
epoch 11/1000    error=0.292237
epoch 12/1000    error=0.291593
epoch 13/1000    error=0.291030
epoch 14/1000    error=0.290531
epoch 15/1000    error=0.290082
epoch 16/1000    error=0.289672
epoch 17/1000    error=0.289295
epoch 18/1000    error=0.288944
epoch 19/1000    error=0.288615
epoch 20/1000    error=0.288305
epoch 21/1000    error=0.288012
epoch 22/1000    error=0.287732
epoch 23/1000    error=0.287466
epoch 24/1000    error=0.287211
epoch 25/1000    error=0.286967
epoch 26/1000    error=0.286732
epoch 27/1000    error=0.286507
epoch 28/1000    error=0.286291
epoch 29/1000    error=0.286083
epoch 30/1000    error=0.285884
epoch 31/1000    error=0.285692
epoch 32/1000    

#### 求解MNIST

我们没有实现卷积层，但这不是问题。我们所需要做的就是重塑我们的数据，使其能够适应一个完全连接的层。

*MNIST数据集由0到9的数字图像组成，形状为28x28x1。目标是预测图片上画的数字。*

In [21]:
import numpy as np
from tensorflow import keras
from keras.datasets import mnist
from keras.utils import np_utils

# load MNIST from server
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# training data : 60000 samples
# reshape and normalize input data
x_train = x_train.reshape(x_train.shape[0], 1, 28*28)
x_train = x_train.astype('float32')
x_train /= 255
# encode output which is a number in range [0,9] into a vector of size 10
# e.g. number 3 will become [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
y_train = np_utils.to_categorical(y_train)

# same for test data : 10000 samples
x_test = x_test.reshape(x_test.shape[0], 1, 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test)

# Network
net = Network()
net.add(FCLayer(28*28, 100))                # input_shape=(1, 28*28)    ;   output_shape=(1, 100)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(100, 50))                   # input_shape=(1, 100)      ;   output_shape=(1, 50)
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(50, 10))                    # input_shape=(1, 50)       ;   output_shape=(1, 10)
net.add(ActivationLayer(tanh, tanh_prime))

# as we didn't implemented mini-batch GD, training will be pretty slow if we update at each iteration on 60000 samples...
net.use(mse, mse_prime)
net.fit(x_train[0:], y_train[0:], epochs=100, learning_rate=0.1)

# test on 3 samples
out = net.predict(x_test[0:3])

epoch 1/100    error=0.041226
epoch 2/100    error=0.020516
epoch 3/100    error=0.016475
epoch 4/100    error=0.014136
epoch 5/100    error=0.012588
epoch 6/100    error=0.011391
epoch 7/100    error=0.010458
epoch 8/100    error=0.009729
epoch 9/100    error=0.009053
epoch 10/100    error=0.008477
epoch 11/100    error=0.007998
epoch 12/100    error=0.007552
epoch 13/100    error=0.007164
epoch 14/100    error=0.006811
epoch 15/100    error=0.006539
epoch 16/100    error=0.006277
epoch 17/100    error=0.006052
epoch 18/100    error=0.005826
epoch 19/100    error=0.005618
epoch 20/100    error=0.005416
epoch 21/100    error=0.005245
epoch 22/100    error=0.005112
epoch 23/100    error=0.005024
epoch 24/100    error=0.004912
epoch 25/100    error=0.004716
epoch 26/100    error=0.004571
epoch 27/100    error=0.004517
epoch 28/100    error=0.004419
epoch 29/100    error=0.004295
epoch 30/100    error=0.004237
epoch 31/100    error=0.004154
epoch 32/100    error=0.004073
epoch 33/100    e

In [22]:
print("predicted values : ")
print(out, end='\n')
print(np.abs(np.around(np.array(out), decimals=0)), end="\n")
print("\ntrue values : ")
print(y_test[0:3])

predicted values : 
[array([[-0.00318634,  0.00119289,  0.00265339,  0.00424661,  0.0390102 ,
         0.00239168,  0.00136381,  0.9949129 ,  0.00135175, -0.01392251]]), array([[-3.52216509e-03,  1.85962447e-03,  9.94601405e-01,
         3.13072808e-03, -2.90730889e-03, -3.88590761e-03,
        -2.71682437e-04,  6.15262275e-03,  1.81434855e-03,
        -1.00280730e-02]]), array([[ 3.03617481e-03,  9.96978622e-01,  4.25644064e-03,
         2.74395676e-03,  1.64727517e-03,  7.31690707e-03,
         1.22548390e-04,  2.28465814e-03,  1.57831000e-05,
        -1.58416103e-02]])]
[[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]

 [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]]

true values : 
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
